In [ ]:
import os
from scipy import ndimage
from scipy.ndimage import rotate
import nibabel as nib
import random
from keras.utils import get_file

In [ ]:
url_1 = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-0.zip"
url_2 = "https://github.com/hasibzunair/3D-image-classification-tutorial/releases/download/v0.2/CT-23.zip"

file_path = "c:\\Users\\maxim\\OneDrive\\Desktop\\Research\\Experiment_1\\datasets\\CT-0.zip"
if os.path.exists(file_path):
    os.remove(file_path)  # Deletes the incomplete file

# Directory where the files will be stored
cache_dir = os.getcwd()  # Current working directory

# Download the first file
filename_1 = get_file("CT-0.zip", url_1, cache_dir=cache_dir)

# Download the second file
filename_2 = get_file("CT-23.zip", url_2, cache_dir=cache_dir)

# Make a directory to store the data
os.makedirs("CT_Data", exist_ok=True)  # Using exist_ok=True to avoid errors if the directory exists

# Unzip the first file
with zipfile.ZipFile(filename_1, "r") as z_fp:
    z_fp.extractall("./CT_Data/")

# Unzip the second file
with zipfile.ZipFile(filename_2, "r") as z_fp:
    z_fp.extractall("./CT_Data/")

In [ ]:
# Function to load a .nii.gz file
def load_nii(file_path):
    return nib.load(file_path).get_fdata()

def save_nii(volume, save_path):
    new_img = nib.Nifti1Image(volume, np.eye(4))  # Assuming the affine matrix is identity
    nib.save(new_img, save_path)

# Function to apply a random rotation to the volume
def random_rotate(volume):
    angle = random.uniform(0, 360)
    return rotate(volume, angle, axes=(0, 1), reshape=False)  # Rotate around the first two axes

def add_contrast(img, factor=1.5):
    mean_intensity = np.mean(img)
    return np.clip((img - mean_intensity) * factor + mean_intensity, 0, 1)

def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]

    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height

    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = load_nii(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [ ]:
def create_multiview_dataset(base_path, ct_folders):
    for ct_folder in ct_folders:
        folder_path = os.path.join(base_path, ct_folder)
        # List all items in the folder and process only files (not directories)
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)

            # Skip hidden files (those starting with a dot) and directories
            if item.startswith('.') or os.path.isdir(item_path):
                continue  # Skip hidden files or directories

            if os.path.isfile(item_path) and item.endswith('.nii.gz'):  # Check if it's a .nii.gz file
                try:
                    # Load the original CT scan image
                    original_volume = process_scan(item_path)

                    # Create subdirectory for each file (based on the file name without extension)
                    study_dir_name = os.path.splitext(item)[0]  # Get the file name without extension
                    study_save_dir = os.path.join(folder_path, study_dir_name)
                    os.makedirs(study_save_dir, exist_ok=True)

                    # Save the original image
                    save_nii(original_volume, os.path.join(study_save_dir, f"{study_dir_name}_original.nii.gz"))

                    # Save the rotated image
                    rotated_volume = random_rotate(original_volume)
                    save_nii(rotated_volume, os.path.join(study_save_dir, f"{study_dir_name}_rotated.nii.gz"))

                    # Save the contrasted image
                    contrast_volume = add_contrast(original_volume)
                    save_nii(contrast_volume, os.path.join(study_save_dir, f"{study_dir_name}_contrast.nii.gz"))

                except Exception as e:
                    print(f"Error processing {item_path}: {e}")

# Directory structure
base_directory = "CT_Data"
ct_folders = ["CT-0", "CT-23"]

# Create multi-view dataset
create_multiview_dataset(base_directory, ct_folders)

In [ ]:
base_path = './CT_Data/CT-0'
base_path_2 = './CT_Data/CT-23'


def delete_files(base_path):

    # Iterate through the files in CT_Data
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)

        # Check if it's a file and if it ends with '.gz'
        if os.path.isfile(item_path) and item.endswith('.gz'):
            os.remove(item_path)  # Delete the file
            print(f"Deleted file: {item_path}")


delete_files(base_path)
delete_files(base_path_2)

In [ ]:
def count_directories(path):
    # List all items in the directory and filter for directories
    return len([d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))])

# Example usage:
path = "CT_Data/CT-23"
path2 = "CT_Data/CT-0"
print(f"Number of directories: {count_directories(path2)}")
print(f"Number of directories: {count_directories(path)}")